# Organizar de tal modo que no toque recalcular matrices!

### Links

* Speeding up Python: https://wiki.python.org/moin/PythonSpeed/PerformanceTips
* PEP8 checking in IPython: http://nbviewer.ipython.org/github/SiggyF/notebooks/blob/master/styleguide.ipynb
* Check if a file exists: http://stackoverflow.com/questions/82831/check-whether-a-file-exists-using-python
* Markdown cheatsheet: https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet
* OpenCV Python Tutorials: https://opencv-python-tutroals.readthedocs.org/en/latest/
* Magic functions https://ipython.org/ipython-doc/dev/interactive/magics.html

### Utilities

```python
[n for n in dir(widgets) if not n.endswith('Widget') and n[0] == n[0].upper() and not n[0] == '_']

nms = dir(cv2) # list of everything in the cv2 module
[m for m in nms if 'PROP' in m]

help(cv2.adaptiveThreshold)
```

In [1]:
# -*- coding: utf-8 -*-
%matplotlib inline

import glob
import os
import re
import time as t


import cv2
import cv2.cv as cv
import IPython
from IPython.display import display
import ipywidgets as widgets
from matplotlib import pyplot as plt
import numpy as np

print "OpenCV version:   {}".format(cv2.__version__)
print "IPython version:  {}".format(IPython.__version__)

OpenCV version:   2.4.11
IPython version:  4.0.0


In [2]:
def change_figsize(w, h):
    fig_size = plt.rcParams["figure.figsize"]
    fig_size[0] = w
    fig_size[1] = h
    plt.rcParams["figure.figsize"] = fig_size
    
def nothing(x):
    pass

In [ ]:
i = cv2.imread('stereo1/sample.png')
img_w = i.shape[1]
img_h = i.shape[0]
del i
RESOLUTION = (img_w, img_h)
grid_w = 7
grid_h = 7
GRID_SIZE = (grid_w, grid_h)
sq_size = 3 # cm
crit = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

objp = np.zeros((grid_w*grid_h,3), np.float32)
objp[:,:2] = np.mgrid[0:grid_w,0:grid_h].T.reshape(-1,2) * sq_size

change_figsize(18, 14)

In [ ]:
imagesL = sorted(glob.glob('stereo1/img/L*.png'))
imagesR = sorted(glob.glob('stereo1/img/R*.png'))

objpoints = []
imgpoints1 = []
imgpoints2 = []

for fnameL, fnameR in zip(imagesL, imagesR):
    imgL = cv2.imread(fnameL)
    imgR = cv2.imread(fnameR)
    grayL = cv2.cvtColor(imgL, cv2.COLOR_BGR2GRAY)
    grayR = cv2.cvtColor(imgR, cv2.COLOR_BGR2GRAY)
    retL, cornersL = cv2.findChessboardCorners(grayL, GRID_SIZE)
    retR, cornersR = cv2.findChessboardCorners(grayR, GRID_SIZE)
        
    if retL is True and retR is True:
        objpoints.append(objp)
        cv2.cornerSubPix(grayL, cornersL, (11,11), (-1,-1), crit)
        cv2.cornerSubPix(grayR, cornersR, (11,11), (-1,-1), crit)
        imgpoints1.append(cornersL)
        imgpoints2.append(cornersR)
        cv2.drawChessboardCorners(imgL, GRID_SIZE, cornersL, retL)
        cv2.drawChessboardCorners(imgR, GRID_SIZE, cornersR, retR)
    print retL, fnameL
    print retR, fnameR
    imgL = cv2.cvtColor(imgL, cv2.COLOR_BGR2RGB)
    imgR = cv2.cvtColor(imgR, cv2.COLOR_BGR2RGB)
    plt.subplot(1,2,1),plt.imshow(imgL)
    plt.subplot(1,2,2),plt.imshow(imgR)
    plt.show()

In [ ]:
retL, mtxL, distL, rvecsL, tvecsL = cv2.calibrateCamera(objpoints, imgpoints1, RESOLUTION)
print mtxL
print distL

retR, mtxR, distR, rvecsR, tvecsR = cv2.calibrateCamera(objpoints, imgpoints2, RESOLUTION)
print mtxR
print distR

In [ ]:
ret, _, _, _, _, R, T, E, F  = cv2.stereoCalibrate(objpoints, imgpoints1, imgpoints2, RESOLUTION)
R1, R2, P1, P2, Q, _, _ = cv2.stereoRectify(mtxL, distL, mtxR, distR, RESOLUTION, R, T, flags=cv2.CALIB_ZERO_DISPARITY)

map1L, map2L = cv2.initUndistortRectifyMap(mtxL, distL, R1, mtxL, RESOLUTION, cv2.CV_16SC2)
map1R, map2R = cv2.initUndistortRectifyMap(mtxR, distR, R2, mtxR, RESOLUTION, cv2.CV_16SC2)

if not os.path.isfile("data/maps.npz"):
    np.savez('data/maps.npz', map1L, map2L, map1R, map2R)

In [ ]:
imagesL = sorted(glob.glob('stereo1/scene/L*.png'))
imagesR = sorted(glob.glob('stereo1/scene/R*.png'))

for fnameL, fnameR in zip(imagesL, imagesR):
    nameL = re.findall(r'[\w\d\-]+\.\w+',fnameL)[0]
    nameR = re.findall(r'[\w\d\-]+\.\w+',fnameR)[0]
    imgL = cv2.imread(fnameL)
    imgR = cv2.imread(fnameR)    
    imgL = cv2.remap(imgL, map1L, map2L, cv2.INTER_CUBIC)
    imgR = cv2.remap(imgR, map1R, map2R, cv2.INTER_CUBIC)
    imgL = cv2.cvtColor(imgL, cv2.COLOR_BGR2GRAY)
    imgR = cv2.cvtColor(imgR, cv2.COLOR_BGR2GRAY)
    cv2.imwrite('stereo1/rect/'+nameL, imgL)
    cv2.imwrite('stereo1/rect/'+nameR, imgR)
    plt.subplot(1,2,1), plt.imshow(imgL, 'gray')
    plt.subplot(1,2,2), plt.imshow(imgR, 'gray')
    plt.show()

In [3]:
with np.load('data/maps.npz') as X:
    map1L, map2L, map1R, map2R = [X[i] for i in ('arr_0', 'arr_1', 'arr_2', 'arr_3')]

In [ ]:
M = 64
w = 3

def stereoSGBM(mindisp=16, sadw=5, p1=42, p2=256, disp12=0, pfiltc=10, unqrat=15, spckw=200, spckr=1):
    imgL = cv2.imread('stereo1/scene/L0296.png')
    imgR = cv2.imread('stereo1/scene/R0296.png')
    imgL = cv2.remap(imgL, map1L, map2L, cv2.INTER_CUBIC)
    imgR = cv2.remap(imgR, map1R, map2R, cv2.INTER_CUBIC)
    
    numdisp = M - mindisp
    w = sadw
    stereo = cv2.StereoSGBM(mindisp, numdisp, sadw, p1, p2, disp12, pfiltc, unqrat, spckw, spckr)
        
    grayL = cv2.cvtColor(imgL,cv2.COLOR_BGR2GRAY)
    grayR = cv2.cvtColor(imgR,cv2.COLOR_BGR2GRAY)
    # grayL = cv2.Laplacian(grayL, cv2.CV_16S, ksize=1, scale=1, delta=0)
    # grayR = cv2.Laplacian(grayR, cv2.CV_16S, ksize=1, scale=1, delta=0)
    # grayL = cv2.convertScaleAbs(grayL, alpha=0.8, beta=100)
    # grayR = cv2.convertScaleAbs(grayR, alpha=0.8, beta=100)
    grayL = cv2.pyrDown(grayL)
    grayR = cv2.pyrDown(grayR)

    disparity = stereo.compute(grayL,grayR)
    change_figsize(9, 7)
    plt.imshow(disparity)
    plt.show()
    change_figsize(18, 14)
    plt.subplot(1,2,1),plt.imshow(grayL, 'gray')
    plt.subplot(1,2,2),plt.imshow(grayR, 'gray')
    # plt.subplot(2,2,3),plt.imshow(dstL, 'gray')
    # plt.subplot(2,2,4),plt.imshow(dstR, 'gray')
    # plt.show()
    
mindisp_slider = widgets.IntSlider(min=0, max=M-16, step=16, value=16, description='minDisparity')
sadw_slider = widgets.IntSlider(min=1, max=21, step=2, value=5, description='SADWindowSize')
P1_slider = widgets.IntSlider(min=0, max=8*w*w, step=1, value=42, description='P1')
P2_slider = widgets.IntSlider(min=0, max=32*w*w, step=1, value=256, description='P2')
disp12_slider = widgets.IntSlider(min=-1, max=5, step=1, value=0, description='disp12MaxDiff')
pfiltc_slider = widgets.IntSlider(min=0, max=63, step=1, value=10, description='preFilterCap')
unqrat_slider = widgets.IntSlider(min=5, max=15, step=1, value=15, description='uniquenessRatio')
spckw_slider = widgets.IntSlider(min=50, max=200, step=1, value=200, description='speckleWindowSize')
spckr_slider = widgets.IntSlider(min=1, max=2, step=1, value=1, description='speckleRange')

w = widgets.interactive(stereoSGBM, mindisp=mindisp_slider, sadw=sadw_slider, p1=P1_slider, p2=P2_slider, disp12=disp12_slider,
                        pfiltc=pfiltc_slider, unqrat=unqrat_slider, spckw=spckw_slider, spckr=spckr_slider)
display(w)

In [ ]:
cap = cv2.VideoCapture(0)
cap2 = cv2.VideoCapture(1)

# cap.set(cv.CV_CAP_PROP_FPS, 2)
# cap2.set(cv.CV_CAP_PROP_FPS, 2)
M = 96
m = 16
n = M - m

with np.load('data/maps.npz') as X:
    map1L, map2L, map1R, map2R = [X[i] for i in ('arr_0', 'arr_1', 'arr_2', 'arr_3')]

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    ret2, frame2 = cap2.read()

    imgL = cv2.remap(frame, map1L, map2L, cv2.INTER_LINEAR)
    imgR = cv2.remap(frame2, map1R, map2R, cv2.INTER_LINEAR)


    stereo = cv2.StereoSGBM(m, n, 1, 8, 32, 3, 41, 14, 200, 1)

    disparity = stereo.compute(imgL,imgR)
    plt.imshow(disparity)
    plt.show()

In [5]:
M = 15

imgL = cv2.imread('stereo1/scene/L0296.png')
imgR = cv2.imread('stereo1/scene/R0296.png')
imgL = cv2.remap(imgL, map1L, map2L, cv2.INTER_CUBIC)
imgR = cv2.remap(imgR, map1R, map2R, cv2.INTER_CUBIC)
imgL = cv2.pyrDown(imgL)
imgR = cv2.pyrDown(imgR)

old_args = []
flag = False

cv2.namedWindow('dmap', cv2.WINDOW_NORMAL)
cv2.namedWindow('img')
cv2.createTrackbar('minDisparity', 'dmap', 0, M-1, nothing)
cv2.createTrackbar('SADWindowSize', 'dmap', 1, 5, nothing)
cv2.createTrackbar('P1', 'dmap', 8, 450, nothing)
cv2.createTrackbar('P2', 'dmap', 32, 2000, nothing)
cv2.createTrackbar('disp12MaxDiff', 'dmap', 0, 3, nothing)
cv2.createTrackbar('preFilterCap', 'dmap', 0, 63, nothing)
cv2.createTrackbar('uniquenessRatio', 'dmap', 5, 15, nothing)
cv2.createTrackbar('speckleWindowSize', 'dmap', 50, 200, nothing)
cv2.createTrackbar('speckleRange', 'dmap', 1, 2, nothing)
switch = '0 : OFF \n1 : ON'
cv2.createTrackbar(switch, 'dmap', 0, 1, nothing)

while True:
    args = [cv2.getTrackbarPos('minDisparity','dmap'),
            cv2.getTrackbarPos('SADWindowSize','dmap'),
            cv2.getTrackbarPos('P1','dmap'),
            cv2.getTrackbarPos('P2','dmap'),
            cv2.getTrackbarPos('disp12MaxDiff','dmap'), 
            cv2.getTrackbarPos('preFilterCap','dmap'),
            cv2.getTrackbarPos('uniquenessRatio','dmap'),
            cv2.getTrackbarPos('speckleWindowSize','dmap'),
            cv2.getTrackbarPos('speckleRange','dmap'),
            cv2.getTrackbarPos(switch,'dmap')
           ]
    
    if args[9] == 0:
        args[2] = 8 * (2*args[1] + 1) ** 2
        args[3] = 32 * (2*args[1] + 1) ** 2
    
    if args != old_args or flag is not True:
        stereo = cv2.StereoSGBM(16*args[0],
                                16* (M - args[0]),
                                2*args[1] + 1,
                                args[2],
                                args[3],
                                args[4],
                                args[5],
                                args[6],
                                args[7],
                                args[8])
        flag = True

    disparity = stereo.compute(imgL,imgR)
    dmap = disparity.astype('uint8')
    cv2.imshow('img', cv2.applyColorMap(-dmap, cv2.COLORMAP_JET))
    
    old_args = args
    
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break
        
cv2.destroyAllWindows()                                                                                               


In [ ]:
help(cv2.createTrackbar)
help(cv2.StereoSGBM)
help(cv2.namedWindow)

In [ ]:
nms = dir(cv2) # list of everything in the cv2 module
[m for m in nms if 'WINDOW' in m]